In [1]:
%cd ~/SSMuLA

/disk2/fli/SSMuLA


In [2]:
%load_ext blackcellmagic

In [3]:
# !python sandbox/esmif/score_log_likelihoods.py sandbox/esmif/5YH2.pdb \
#   sandbox/esmif/5YH2_mutated_seqs.fasta --chain C \
#   --outpath sandbox/esmif/5YH2_mutated_seqs_scores.csv

In [4]:
import pandas as pd

In [5]:
import warnings

from Bio.PDB import PDBParser
from Bio.SeqUtils import seq1
from Bio import SeqIO, pairwise2
from Bio.pairwise2 import format_alignment

from SSMuLA.landscape_global import LIB_INFO_DICT

warnings.filterwarnings("ignore")


def pdb2seq(pdb_file_path: str, chain_id: str = "A") -> str:

    """
    A function for extracting chain in string format from pdb

    Args:
    - pdb_file_path: str,
    - chain_id: str = "A"
    """

    chains = {
        record.id: record.seq for record in SeqIO.parse(pdb_file_path, "pdb-atom")
    }

    return str(chains[[chain for chain in chains.keys() if chain_id in chain][0]])


def find_missing_str(longer: str, shorter: str) -> [str, str]:
    """
    A function for finding the missing part of a string

    Args:
    - longer: str, longer string
    - shorter: str, shorter string

    Returns:
    - part_before: str, part of the longer string before the shorter
    - part_after: str, part of the longer string after the shorter
    """
    # Find the start index of the shorter in the longer string
    start_index = longer.find(shorter)

    # If the shorter is not found need to do alingment to figure out
    if start_index == -1:
        
        return "", ""

    # Find the end index of the shorter
    end_index = start_index + len(shorter)

    # Extract parts of the longer string that are not the shorter
    part_before = longer[:start_index]
    part_after = longer[end_index:]

    return part_before, part_after

def mut_csv2fasta(lib: str, ev_esm_dir: str = "ev_esm2") -> None:
    """
    A function for converting mutation csv to fasta

    Args:
    - lib: str, path to mutation csv
    - ev_esm_dir: str = "ev_esm2"
    """

    csv_path = f"{ev_esm_dir}/{lib}/{lib}.csv"

    if "TrpB" in lib:
        protein = "TrpB"
    else:
        protein = lib

    seq = SeqIO.read(f"data/{protein}/{protein}.fasta", "fasta").seq

    if lib == "DHFR":
        seq = str(seq.translate())
    else:
        seq = str(seq)

    pdb_seq = pdb2seq(f"data/{protein}/{protein}.pdb", "A")

    df = pd.read_csv(csv_path)

    for col in ["muts", "seq"]:
        if col not in df.columns:
            raise ValueError(f"{col} column not found")

    fasta = csv_path.replace(".csv", ".fasta")

    print(f"Writing to {fasta}...")

    # pdb has more than fasta should only be for dhfr
    if len(seq) < len(pdb_seq):
        print("PDB seq is longer than fasta")
        part_before, part_after = find_missing_str(longer=pdb_seq, shorter=seq)
        with open(fasta, "w") as f:
            for mut, seq in zip(df["muts"].values, df["seq"].values):
                f.write(f">{mut}\n{part_before+seq+part_after}\n")
    elif len(seq) == len(pdb_seq):
        print("PDB seq length is equal to fasta")
        with open(fasta, "w") as f:
            for mut, seq in zip(df["muts"].values, df["seq"].values):
                f.write(f">{mut}\n{seq}\n")
    else:
        print("Fasta seq is longer than PDB")
        part_before, part_after = find_missing_str(longer=seq, shorter=pdb_seq)
        with open(fasta, "w") as f:
            for mut, seq in zip(df["muts"].values, df["seq"].values):
                f.write(f">{mut}\n{seq[len(part_before):len(seq)-len(part_after)]}\n")


/disk2/fli/miniconda3/envs/SSMuLA/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [23]:
protein = "TEV"
lib = "TEV"

seq = SeqIO.read(f"data/{protein}/{protein}.fasta", "fasta").seq

seq = str(seq)

pdb_seq = pdb2seq(f"data/{protein}/{protein}.pdb", "A")

global_alignments = pairwise2.align.globalxx(seq, pdb_seq)

print(len(seq), len(pdb_seq))


233 230


In [24]:
df = pd.read_csv(f"ev_esm2/{lib}/{lib}.csv")

In [25]:
df

,AAs,AA1,AA2,AA3,AA4,fitness,active,muts,n_mut,seq,combo,pos,esm_score
0,IEMG,I,E,M,G,-0.054598,False,T146I:D148E:H167M:S170G,4,MFKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFR...,"['I', 'E', 'M', 'G']","[146, 148, 167, 170]",-109.408524
1,HMIR,H,M,I,R,0.003133,False,T146H:D148M:H167I:S170R,4,MFKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFR...,"['H', 'M', 'I', 'R']","[146, 148, 167, 170]",-109.191965
2,HNED,H,N,E,D,-0.009535,False,T146H:D148N:H167E:S170D,4,MFKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFR...,"['H', 'N', 'E', 'D']","[146, 148, 167, 170]",-107.576699
3,HWFP,H,W,F,P,-0.057285,False,T146H:D148W:H167F:S170P,4,MFKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFR...,"['H', 'W', 'F', 'P']","[146, 148, 167, 170]",-102.012828
4,HTFN,H,T,F,N,-0.007008,False,T146H:D148T:H167F:S170N,4,MFKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFR...,"['H', 'T', 'F', 'N']","[146, 148, 167, 170]",-104.148911
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159127,CKLF,C,K,L,F,-0.035015,False,T146C:D148K:H167L:S170F,4,MFKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFR...,"['C', 'K', 'L', 'F']","[146, 148, 167, 170]",-102.514650
159128,DHRT,D,H,R,T,0.045136,True,T146D:D148H:H167R:S170T,4,MFKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFR...,"['D', 'H', 'R', 'T']","[146, 148, 167, 170]",-93.501041
159129,CGGF,C,G,G,F,0.013294,False,T146C:D148G:H167G:S170F,4,MFKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFR...,"['C', 'G', 'G', 'F']","[146, 148, 167, 170]",-83.973839
159130,CVSV,C,V,S,V,-0.007239,False,T146C:D148V:H167S:S170V,4,MFKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFR...,"['C', 'V', 'S', 'V']","[146, 148, 167, 170]",-80.439535


In [26]:
seq

'MFKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNGTLLVQSLHGVFKVKNTTTLQQHLIDGRDMIIIRMPKDFPPFPQKLKFREPQREERICLVTTNFQTKSMSSMVSDTSCTFPSSDGIFWKHWIQTKDGQCGSPLVSTRDGFIVGIHSASNFTNTNNYFTSVPKNFMELLTNQEAQQWVSGWRLNADSVLWGGHKVFMVKPEEPFQPVKEATQLMN'

In [27]:
pdb_seq

'GHHHHHHHXGESLFKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNGTLLVQSLHGVFKVKNTTTLQQHLIDGRDMIIIRMPKDFPPFPQKLKFREPQREERICLVTTNFQTKSMSSMVSDTSCTFPSSDGIFWKHWIQTKDGQCGSPLVSTRDGFIVGIHSASNFTNTNNYFTSVPKNFMELLTNQEAQQWVSGWRLNADSVLWGGHKVFMDKP'

In [28]:
print(format_alignment(*pairwise2.align.globalms(seq, pdb_seq, 2, -1, -0.5, -0.1)[0]))

M-------------FKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNGTLLVQSLHGVFKVKNTTTLQQHLIDGRDMIIIRMPKDFPPFPQKLKFREPQREERICLVTTNFQTKSMSSMVSDTSCTFPSSDGIFWKHWIQTKDGQCGSPLVSTRDGFIVGIHSASNFTNTNNYFTSVPKNFMELLTNQEAQQWVSGWRLNADSVLWGGHKVFMV-KPEEPFQPVKEATQLMN
              ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||  ||               
-GHHHHHHHXGESLFKGPRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNGTLLVQSLHGVFKVKNTTTLQQHLIDGRDMIIIRMPKDFPPFPQKLKFREPQREERICLVTTNFQTKSMSSMVSDTSCTFPSSDGIFWKHWIQTKDGQCGSPLVSTRDGFIVGIHSASNFTNTNNYFTSVPKNFMELLTNQEAQQWVSGWRLNADSVLWGGHKVFM-DKP---------------
  Score=426.9



In [12]:
def alignmutseq2pdbseq(mut_seq: str, pdb_seq: str) -> list[int]:
    """
    A function for aligning mutation sequence to pdb sequence and
    return the indices of the aligned sequence so that the mutation
    sequence can be trimmed to the lenght of the pdb sequence

    Args:
    - mut_seq: str, mutation sequence
    - pdb_seq: str, pdb sequence
    """

    # Define a custom scoring function so that X is aligned with anything
    def custom_match_function(x, y):
        if x == "X" or y == "X":
            return 2  # High score for aligning X with anything
        elif x == y:
            return 2  # Match score
        else:
            return -1  # Mismatch score

    _, aligned_pdb_seq, _, _, _ = pairwise2.align.globalcs(
        mut_seq, pdb_seq, custom_match_function, -0.5, -0.1
    )[0]

    return [
        aligned_pdb_seq.find(aligned_pdb_seq.replace("-", "")[:1]),
        aligned_pdb_seq.rfind(aligned_pdb_seq.replace("-", "")[-1]),
    ]

In [40]:

# Define a custom scoring function
def custom_match_function(x, y):
    if x == 'X' or y == 'X':
        return 2  # High score for aligning X with anything
    elif x == y:
        return 2  # Match score
    else:
        return -1  # Mismatch score


In [24]:
print(format_alignment(*pairwise2.align.globalcs(seq, pdb_seq, custom_match_function, -0.5, -0.1)[0]))

MNTINIAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEMGEARFRKMFERQLKAGEVADNAAAKPLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIKTTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKKAFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQDSETIELAPEYAEAIATRAGALAGISPMFQPCVVPPKPWTGITGGGYWANGRRPLALVRTHSKKALMRYEDVYMPEVYKAINIAQNTAWKINKKVLAVANVITKWKHCPVEDIPAIEREELPMKPEDIDMNPEALTAWKRAAAAVYRKDKARKSRRISLEFMLEQANKFANHKAIWFPYNMDWRGRVYAVSMFNPQGNDMTKGLLTLAKGKPIGKEGYYWLKIHGANCAGVDKVPFPERIKFIEENHENIMACAKSPLENTWWAEQDSPFCFLAFCFEYAGVQHHGLSYNCSLPLAFDGSCSGIQHFSAMLRDEVGGRAVNLLPSETVQDIYGIVAKKVNEILQADAINGTDNEVVTVTDENTGEISEKVKLGTKALAGQWLAYGVTRSVTKRSVMTLAYGSKEFGFRQQVLEDTIQPAIDSGKGLMFTQPNQAAGYMAKLIWESVSVTVVAAVEAMNWLKSAAKLLAAEVKDKKTGEILRKRCAVHWVTPDGFPVWQEYKKPIQTRLNLMFLGQFRLQPTINTNKDSEIDAHKQESGIAPNFVHSQDGSHLRKTVVWAHEKYGIESFALIHDSFGTIPADAANLFKAVRETMVDTYESCDVLADFYDQFADQLHESQLDKMPALPAKGNLNLRDILESDFAFA
     ||||||||||||||||||||||||||||||||||||||||||||||||||................|||||||||||||||||||||||||||||||||||||||||||||

In [49]:
# Extract the aligned sequences
aligned_seq1, aligned_seq2, score, begin, end = pairwise2.align.globalcs(seq, pdb_seq, custom_match_function, -0.5, -0.1)[0]

# Determine the start and end positions of the aligned regions (ignoring gaps)
start_index_seq1 = aligned_seq1.find(aligned_seq1.replace('-', '')[:1])
end_index_seq1 = aligned_seq1.rfind(aligned_seq1.replace('-', '')[-1])

start_index_seq2 = aligned_seq2.find(aligned_seq2.replace('-', '')[:1])
end_index_seq2 = aligned_seq2.rfind(aligned_seq2.replace('-', '')[-1])

# Print the start and end indices
print(f"Aligned region in seq1: Start = {start_index_seq1}, End = {end_index_seq1}")
print(f"Aligned region in seq2: Start = {start_index_seq2}, End = {end_index_seq2}")


Aligned region in seq1: Start = 0, End = 882
Aligned region in seq2: Start = 5, End = 882


In [55]:
# Step 1: Find all indices of 'X' in pdb_seq
x_indices = [i for i, letter in enumerate(aligned_seq2) if letter == 'X']
x_indices

[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]

In [56]:
# Step 2: Modify the original seq by replacing characters at the found indices with 'X'
seq_list = list(seq)  # Convert the sequence to a list to allow mutation
for idx in x_indices:
    seq_list[idx] = 'X'

# Convert the modified list back to a string
modified_seq = ''.join(seq_list)
modified_seq

'MNTINIAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEMGEARFRKMFXXXXXXXXXXXXXXXXPLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIKTTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKKAFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQDSETIELAPEYAEAIATRAGALAGISPMFQPCVVPPKPWTGITGGGYWANGRRPLALVRTHSKKALMRYEDVYMPEVYKAINIAQNTAWKINKKVLAVANVITKWKHCPVEDIPAIEREELPMKPEDIDMNPEALTAWKRAAAAVYRKDKARKSRRISLEFMLEQANKFANHKAIWFPYNMDWRGRVYAVSMFNPQGNDMTKGLLTLAKGKPIGKEGYYWLKIHGANCAGVDKVPFPERIKFIEENHENIMACAKSPLENTWWAEQDSPFCFLAFCFEYAGVQHHGLSYNCSLPLAFDGSCSGIQHFSAMLRDEVGGRAVNLLPSETVQDIYGIVAKKVNEILQADAINGTDNEVVTVTDENTGEISEKVKLGTKALAGQWLAYGVTRSVTKRSVMTLAYGSKEFGFRQQVLEDTIQPAIDSGKGLMFTQPNQAAGYMAKLIWESVSVTVVAAVEAMNWLKSAAKLLAAEVKDKKTGEILRKRCAVHWVTPDGFPVWQEYKKPIQTRLNLMFLGQFRLQPTINTNKDSEIDAHKQESGIAPNFVHSQDGSHLRKTVVWAHEKYGIESFALIHDSFGTIPADAANLFKAVRETMVDTYESCDVLADFYDQFADQLHESQLDKMPALPAKGNLNLRDILESDFAFA'

In [30]:
from Bio import PDB

from Bio import PDB

def chop_pdb(input_pdb: str, output_pdb: str, start_resid: int, end_resid: int, chain_id: str):
    # Initialize the parser and structure
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("structure", input_pdb)
    
    # Initialize the writer
    io = PDB.PDBIO()

    # Define a select class to filter the residues in the specific chain
    class ResidueSelect(PDB.Select):
        def accept_residue(self, residue):
            # Only accept residues in the specified chain with a residue ID greater than or equal to start_resid
            if residue.parent.id == chain_id and residue.id[1] >= start_resid and residue.id[1] <= end_resid:
                return True
            return False

    # Save the chopped structure to the output file
    io.set_structure(structure)
    io.save(output_pdb, ResidueSelect())

    print(f"Saved chopped structure starting from residue {start_resid} in chain {chain_id} to {output_pdb}")


In [ ]:
# Example usage
input_pdb = "/disk2/fli/SSMuLA/data/T7/T7.pdb"  # Replace with your input PDB file
output_pdb = "/disk2/fli/SSMuLA/data/T7/T7_processed.pdb"  # Replace with your desired output PDB file
start_resid = 71  # Replace with the starting residue ID you want

chop_pdb(input_pdb, output_pdb, start_resid, "A")


In [32]:
input_pdb = "/disk2/fli/SSMuLA/data/TEV/TEV.pdb"  # Replace with your input PDB file
output_pdb = "/disk2/fli/SSMuLA/data/TEV/TEV_processed.pdb"  # Replace with your desired output PDB file
start_resid = 5  # Replace with the starting residue ID you want

chop_pdb(input_pdb, output_pdb, start_resid, 218, "A")

Saved chopped structure starting from residue 5 in chain A to /disk2/fli/SSMuLA/data/TEV/TEV_processed.pdb


In [45]:
begin, end

(0, 883)

In [50]:
aligned_seq1[start_index_seq2:end_index_seq2+1]

'IAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEMGEARFRKMFERQLKAGEVADNAAAKPLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIKTTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKKAFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQDSETIELAPEYAEAIATRAGALAGISPMFQPCVVPPKPWTGITGGGYWANGRRPLALVRTHSKKALMRYEDVYMPEVYKAINIAQNTAWKINKKVLAVANVITKWKHCPVEDIPAIEREELPMKPEDIDMNPEALTAWKRAAAAVYRKDKARKSRRISLEFMLEQANKFANHKAIWFPYNMDWRGRVYAVSMFNPQGNDMTKGLLTLAKGKPIGKEGYYWLKIHGANCAGVDKVPFPERIKFIEENHENIMACAKSPLENTWWAEQDSPFCFLAFCFEYAGVQHHGLSYNCSLPLAFDGSCSGIQHFSAMLRDEVGGRAVNLLPSETVQDIYGIVAKKVNEILQADAINGTDNEVVTVTDENTGEISEKVKLGTKALAGQWLAYGVTRSVTKRSVMTLAYGSKEFGFRQQVLEDTIQPAIDSGKGLMFTQPNQAAGYMAKLIWESVSVTVVAAVEAMNWLKSAAKLLAAEVKDKKTGEILRKRCAVHWVTPDGFPVWQEYKKPIQTRLNLMFLGQFRLQPTINTNKDSEIDAHKQESGIAPNFVHSQDGSHLRKTVVWAHEKYGIESFALIHDSFGTIPADAANLFKAVRETMVDTYESCDVLADFYDQFADQLHESQLDKMPALPAKGNLNLRDILESDFAFA'

In [44]:
aligned_seq2

'-----IAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEMGEARFRKMFXXXXXXXXXXXXXXXXPLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIKTTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKKAFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQDSETIELAPEYAEAIATRAGALAGISPMFQPCVVPPKPWTGITGGGYWANGRRPLALVRTHSKKALMRYEDVYMPEVYKAINIAQNTAWKINKKVLAVANVITKWKHCPVEDIPAIEREELPMKPEDIDMNPEALTAWKRAAAAVYRKDKARKSRRISLEFMLEQANKFANHKAIWFPYNMDWRGRVYAVSMFNPQGNDMTKGLLTLAKGKPIGKEGYYWLKIHGANCAGVDKVPFPERIKFIEENHENIMACAKSPLENTWWAEQDSPFCFLAFCFEYAGVQHHGLSYNCSLPLAFDGSCSGIQHFSAMLRDEVGGRAVNLLPSETVQDIYGIVAKKVNEILQADAINGTDNEVVTVTDENTGEISEKVKLGTKALAGQWLAYGVTRSVTKRSVMTLAYGSKEFGFRQQVLEDTIQPAIDSGKGLMFTQPNQAAGYMAKLIWESVSVTVVAAVEAMNWLKSAAKLLAAEVKDKKTGEILRKRCAVHWVTPDGFPVWQEYKKPIQTRLNLMFLGQFRLQPTINTNKDSEIDAHKQESGIAPNFVHSQDGSHLRKTVVWAHEKYGIESFALIHDSFGTIPADAANLFKAVRETMVDTYESCDVLADFYDQFADQLHESQLDKMPALPAKGNLNLRDILESDFAFA'

In [16]:
len("PLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIKTTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKKAFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQDSETIELAPEYAEAIATRAGALAGISPMFQPCVVPPKPWTGITGGGYWANGRRPLALVRTHSKKALMRYEDVYMPEVYKAINIAQNTAWKINKKVLAVANVITKWKHCPVEDIPAIEREELPMKPEDIDMNPEALTAWKRAAAAVYRKDKARKSRRISLEFMLEQANKFANHKAIWFPYNMDWRGRVYAVSMFNPQGNDMTKGLLTLAKGKPIGKEGYYWLKIHGANCAGVDKVPFPERIKFIEENHENIMACAKSPLENTWWAEQDSPFCFLAFCFEYAGVQHHGLSYNCSLPLAFDGSCSGIQHFSAMLRDEVGGRAVNLLPSETVQDIYGIVAKKVNEILQADAINGTDNEVVTVTDENTGEISEKVKLGTKALAGQWLAYGVTRSVTKRSVMTLAYGSKEFGFRQQVLEDTIQPAIDSGKGLMFTQPNQAAGYMAKLIWESVSVTVVAAVEAMNWLKSAAKLLAAEVKDKKTGEILRKRCAVHWVTPDGFPVWQEYKKPIQTRLHLMFLGQFTLYPTINTNKDSEIDAHKQESGIAPNFVHSQDGSHLRKTVVWAHEKYGIESFALIHDSFGTIPADAANLFKAVRETMVDTYESCDVLADFYDQFADQLHESQLDKMPALPAKGNLNLRDILESDFAF")

811

In [26]:
from Bio.Align import substitution_matrices, PairwiseAligner

aligner = PairwiseAligner()

# Use the default substitution matrix and modify it for the wildcard 'X'
matrix = substitution_matrices.load("BLOSUM62")

# Allow 'X' to align with anything with no penalty
for residue in matrix.alphabet:
    matrix['X', residue] = 2  # High score for aligning 'X' with anything
    matrix[residue, 'X'] = 2

aligner.substitution_matrix = matrix
aligner.mode = 'global'

# Perform the alignment

# Print the best alignment
print(aligner.align(seq, pdb_seq)[0])


target            0 MNTINIAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEMGEARFRKMFERQLK
                  0 ---|--|||||||||||||||||||||||||||||||||||||||||||||||||.....
query             0 ---I--AKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEMGEARFRKMFXXXXX

target           60 AGEVADNAAAKPLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIK
                 60 ...........|||||||||||||||||||||||||||||||||||||||||||||||||
query            55 XXXXXXXXXXXPLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIK

target          120 TTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKK
                120 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
query           115 TTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKK

target          180 AFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQD
                180 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
query           175 AFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQD

target          240 SETI

In [35]:
part_before, part_after = find_missing_str(longer=seq, shorter=pdb_seq)

In [36]:
part_before

''

In [37]:
part_after

''

In [48]:

    if "TrpB" in lib:
        protein = "TrpB"
    else:
        protein = lib

    print(f"data/{protein}/{protein}.pdb")

    seq = SeqIO.read(f"data/{protein}/{protein}.fasta", "fasta").seq
    if lib == "DHFR":
        seq = str(seq.translate())
    else:
        seq = str(seq)

    pdb_seq = pdb2seq(f"data/{protein}/{protein}.pdb", "A")

    global_alignments = pairwise2.align.globalxx(seq, pdb_seq)

    print(len(seq), len(pdb_seq))
    
    # pdb has more than fasta should only be for dhfr
    if len(seq) < len(pdb_seq):
        
        
        
        
        
        

DHFR
data/DHFR/DHFR.pdb
73 159
MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHTWESIGRPLPGRKNIILSSQPGTDDRVT MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHTWESIGRPLPGRKNIILSSQPGTDDRVTWVKSVDEAIAACGDVPEIMVIGGGRVYEQFLPKAQKLYLTHIDAEVEGDTHFPDYEPDDWESVFSEFHDADAQNSHSYCFEILERR
Global Alignment:
MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHTWESIGRPLPGRKNIILSSQPGTDDRVT--------------------------------------------------------------------------------------
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||                                                                                      
MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHTWESIGRPLPGRKNIILSSQPGTDDRVTWVKSVDEAIAACGDVPEIMVIGGGRVYEQFLPKAQKLYLTHIDAEVEGDTHFPDYEPDDWESVFSEFHDADAQNSHSYCFEILERR
  Score=73

ParD2
data/ParD2/ParD2.pdb
91 87
MANVEKMSVAVTPQQAAVMREAVEAGEYATASEIVREAVRDWLAKRELREAEAERLRKAWIEGLESGPFAPFDIEDIKQKARSRLVDAIKK NVEKMSVAVTPQQAAVMREAVEAGEYATASEIVREAVRDWLAKRELREAEAERLRKAWIEGLESGPFAPFDIEDIKQKARSRLVDAI
Global Align

In [55]:
global_alignments[0].seqA

'MKGYFGPYGGQYVPEILMGALEELEAAYEGIMKDESFWKEFNDLLRDYAGRPTPLYFARRLSEKYGARVYLKREDLLHTGAHKINNAIGQVLLAKLMGKTRIIAETGAGQHGVATATAAALFGMECVIYMGEEDTIRQKLNVERMKLLGAKVVPVKSGSRTLKDAIDEALRDWITNLQTTYYVFGSVVGPHPYPIIVRNFQKVIGEETKKQIPEKEGRLPDYIVACVSGGSNAAGIFYPFIDSGVKLIGVEAGGEGLETGKHAASLLKGKIGYLHGSKTFVLQDDWGQVQVSHSVSAGLDYSGVGPEHAYWRETGKVLYDAVTDEEALDAFIELSRLEGIIPALESSHALAYLKKINIKGKVVVVNLSGRGDKDLESVLNHPYVRERIRL'

In [56]:
global_alignments[0].seqB

'--GYFGPYGGQYVPEILMGALEELEAAYEGIMKDESFWKEFNDLLRDYAGRPTPLYFARRLSEKYGARVYLKREDLLHTGAHKINNAIGQVLLAKLMGKTRIIAETGAGQHGVATATAAALFGMECVIYMGEEDTIRQKLNVERMKLLGAKVVPVKSGSRTLKDAIDEALRDWITNLQTTYYVFGSVVGPHPYPIIVRNFQKVIGEETKKQIPEKEGRLPDYIVACVSGGSNAAGIFYPFIDSGVKLIGVEAGGEGLETGKHAASLLKGKIGYLHGSKTFVLQDDWGQVQVSHSVSAGLDYSGVGPEHAYWRETGKVLYDAVTDEEALDAFIELSRLEGIIPALESSHALAYLKKINIKGKVVVVNLSGRGDKDLESVLNHPYVRERIRL'

In [1]:
def find_missing_str(longer: str, shorter: str) -> [str, str]:
    """
    A function for finding the missing part of a string

    Args:
    - longer: str, longer string
    - shorter: str, shorter string

    Returns:
    - part_before: str, part of the longer string before the shorter
    - part_after: str, part of the longer string after the shorter
    """
    # Find the start index of the shorter in the longer string
    start_index = longer.find(shorter)

    # If the shorter is not found, return the longer string as the "missing" part
    if start_index == -1:
        return longer, ""

    # Find the end index of the shorter
    end_index = start_index + len(shorter)

    # Extract parts of the longer string that are not the shorter
    part_before = longer[:start_index]
    part_after = longer[end_index:]

    return part_before, part_after

In [16]:
ev_esm_dir = "ev_esm2"
csv_path = f"{ev_esm_dir}/{lib}/{lib}.csv"

if "TrpB" in lib:
    protein = "TrpB"
else:
    protein = lib
    
seq = SeqIO.read(f"data/{protein}/{protein}.fasta", "fasta").seq
if lib == "DHFR":
    seq = str(seq.translate())
else:
    seq = str(seq)

pdb_seq = pdb2seq(f"data/{protein}/{protein}.pdb", "A")

global_alignments = pairwise2.align.globalxx(seq, pdb_seq)

df = pd.read_csv(df_path)

for col in ["muts", "seq"]:
    if col not in df.columns:
        raise ValueError(f"{col} column not found")

fasta = csv_path.replace(".csv", ".fasta")

# pdb has more than fasta should only be for dhfr
if len(seq) < len(pdb_seq):
    part_before, part_after = find_missing_str(longer=pdb_seq, shorter=seq)
    with open(fasta, "w") as f:
        for mut, seq in zip(df["muts"].values, df["seq"].values):
            f.write(f">{mut}\n{part_before+seq+part_after}\n")
elif len(seq) == len(pdb_seq):
    with open(fasta, "w") as f:
        for mut, seq in zip(df["muts"].values, df["seq"].values):
            f.write(f">{mut}\n{seq}\n")
else:
    part_before, part_after = find_missing_str(longer=seq, shorter=pdb_seq)
    with open(fasta, "w") as f:
        for mut, seq in zip(df["muts"].values, df["seq"].values):
            f.write(f">{mut}\n{seq[len(part_before):len(seq)-len(part_after)]}\n")



NameError: name 'df_path' is not defined

In [14]:
mut, seq = df["muts"].values[0], df["seq"].values[0]

In [38]:
seq[len(part_before):len(seq)-len(part_after)]

'MNTINIAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEMGEARFRKMFERQLKAGEVADNAAAKPLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIKTTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKKAFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQDSETIELAPEYAEAIATRAGALAGISPMFQPCVVPPKPWTGITGGGYWANGRRPLALVRTHSKKALMRYEDVYMPEVYKAINIAQNTAWKINKKVLAVANVITKWKHCPVEDIPAIEREELPMKPEDIDMNPEALTAWKRAAAAVYRKDKARKSRRISLEFMLEQANKFANHKAIWFPYNMDWRGRVYAVSMFNPQGNDMTKGLLTLAKGKPIGKEGYYWLKIHGANCAGVDKVPFPERIKFIEENHENIMACAKSPLENTWWAEQDSPFCFLAFCFEYAGVQHHGLSYNCSLPLAFDGSCSGIQHFSAMLRDEVGGRAVNLLPSETVQDIYGIVAKKVNEILQADAINGTDNEVVTVTDENTGEISEKVKLGTKALAGQWLAYGVTRSVTKRSVMTLAYGSKEFGFRQQVLEDTIQPAIDSGKGLMFTQPNQAAGYMAKLIWESVSVTVVAAVEAMNWLKSAAKLLAAEVKDKKTGEILRKRCAVHWVTPDGFPVWQEYKKPIQTRLNLMFLGQFRLQPTINTNKDSEIDAHKQESGIAPNFVHSQDGSHLRKTVVWAHEKYGIESFALIHDSFGTIPADAANLFKAVRETMVDTYESCDVLADFYDQFADQLHESQLDKMPALPAKGNLNLRDILESDFAFA'

In [31]:
len(part_before)

0

In [33]:
len(seq), len(part_after), len(seq)-len(part_after)

(883, 883, 0)

In [ ]:

def csv2fasta(csv: str) -> None:
    """
    A function for converting a csv file to a fasta file
    ie /disk2/fli/SSMuLA/ev_esm2/DHFR/DHFR.csv

    """
    df = pd.read_csv(csv)

    for col in ["muts", "seq"]:
        if col not in df.columns:
            raise ValueError(f"{col} column not found")

    fasta = csv.replace(".csv", ".fasta")
    with open(fasta, "w") as f:
        for mut, seq in zip(df["muts"].values, df["seq"].values):
            f.write(f">{mut}\n{seq}\n")


In [6]:
from Bio import PDB

def modify_chain_id(input_pdb, output_pdb, original_chain_id='B', new_chain_id='A'):
    # Initialize the PDB parser
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure('structure', input_pdb)
    
    # Loop through all models, chains, and residues
    for model in structure:
        for chain in model:
            if chain.id == original_chain_id:
                chain.id = new_chain_id
    
    # Save the modified structure
    io = PDB.PDBIO()
    io.set_structure(structure)
    io.save(output_pdb)
    print(f"Saved modified PDB file as {output_pdb}")

# Example usage
input_pdb = '/disk2/fli/SSMuLA/coves_data/TEV2/TEV2.pdb'  # Replace with your input PDB file path
output_pdb = '/disk2/fli/SSMuLA/coves_data/TEV2/TEV2.pdb'  # Replace with your desired output PDB file path

modify_chain_id(input_pdb, output_pdb, original_chain_id='B', new_chain_id='A')


Saved modified PDB file as /disk2/fli/SSMuLA/coves_data/TEV2/TEV2.pdb
